In [1]:
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta
import pickle
import re

In [5]:
c = pd.DataFrame()
c = pd.read_feather("/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source/10_28_2020")
c.head()

,Title,Company,Location,Link,Salary,Date,Description
0,Identity and Access Management Support Analyst...,Solu Technology Partners,None,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,10/28/2020,Title: Identity and Access Management Support ...
1,Lawson/Infor Systems Analyst,Arch Staffing & Consulting,None,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"$80,000 - $90,000 a year",10/28/2020,Location: Spokane WA (remote during COVID)\nSa...
2,Forecast Analyst,Nexeo Plastics,"Dublin, OH 43016",https://www.indeed.com/rc/clk?jk=af3b81fe16665...,None,10/28/2020,"Date: Oct 28, 2020\nLocation: Dublin, OH, US, ..."
3,Production Support Analyst,Accenture,"Columbus, OH",https://www.indeed.com/rc/clk?jk=a19089fc3d9a2...,None,10/28/2020,Accenture Flex offers you the flexibility of l...
4,Corrections Data Analyst II,"JPMorgan Chase Bank, N.A.","Columbus, OH",https://www.indeed.com/rc/clk?jk=03a530d9989ff...,None,10/28/2020,Position Overview\nCCB Remediation and Correct...


In [6]:
import os

In [9]:
list = os.listdir("/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source") # dir is your directory path
number_files = len(list)
number_files

5

In [13]:
import glob
dir_name = "/Users/huilingchen/Documents/GitHub/indeed_web_scraper/source"
merged_df = pd.DataFrame()
for filename in os.listdir(dir_name):
    temp_df = pd.read_feather(filename)
    merged_df.append(temp_df)
merged_df.size()

ArrowInvalid: Not a Feather V1 or Arrow IPC file